# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [65]:
# import libraries
import pandas as pd
import numpy as np
import nltk
from pprint import pprint
nltk.download(['punkt', 'wordnet'])
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import TruncatedSVD
import pickle
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier



[nltk_data] Downloading package punkt to /Users/sabrina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sabrina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('disaster_response', engine)
X = df['message']  # Message Column
Y = df.iloc[:, 4:] # Classification label

In [3]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [30]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [31]:
pipeline = Pipeline([
        ('countVect', CountVectorizer(tokenizer=tokenize)),
        ('trasform', TfidfTransformer()),
        ('classifier', MultiOutputClassifier(RandomForestClassifier(random_state = 42)))
    ])


# Look at parameters used by current pipeline
print('Parameters currently in use:\n')
pprint(pipeline.get_params())



Parameters currently in use:

{'classifier': MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)),
 'classifier__estimator': RandomForestClassifier(random_state=42),
 'classifier__estimator__bootstrap': True,
 'classifier__estimator__ccp_alpha': 0.0,
 'classifier__estimator__class_weight': None,
 'classifier__estimator__criterion': 'gini',
 'classifier__estimator__max_depth': None,
 'classifier__estimator__max_features': 'auto',
 'classifier__estimator__max_leaf_nodes': None,
 'classifier__estimator__max_samples': None,
 'classifier__estimator__min_impurity_decrease': 0.0,
 'classifier__estimator__min_impurity_split': None,
 'classifier__estimator__min_samples_leaf': 1,
 'classifier__estimator__min_samples_split': 2,
 'classifier__estimator__min_weight_fraction_leaf': 0.0,
 'classifier__estimator__n_estimators': 100,
 'classifier__estimator__n_jobs': None,
 'classifier__estimator__oob_score': False,
 'classifier__estimator__random_state': 42,
 'classifier__estimator

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 25)
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(steps=[('countVect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fd10feb18b0>)),
                ('trasform', TfidfTransformer()),
                ('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [47]:
def performace_metric(model, X_test, y_true):
    '''
    Function to generate classification report on the model
    Input: Model, test set ie X_test & y_test
    Output: Prints the Classification report
    '''
    accuracy_list = []
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_true):
        accuracy = accuracy_score(y_true[col], y_pred[:, i])
        print('*****', col, '*****')
        print("Accuracy: {:.3f}\t\t% Precision: {:.3f}\t% Recall: {:.3f}\t\t% F1_score: {:.3f}".format(
            accuracy,
            precision_score(y_true[col], y_pred[:, i], average='weighted'),
            recall_score(y_true[col], y_pred[:, i], average='weighted'),
            f1_score(y_true[col], y_pred[:, i], average='weighted')))
        print('---------------------------------------------------------')
        accuracy_list.append(accuracy)
    return accuracy_list
                
base_metric = performace_metric(pipeline, X_test, y_test)



***** related *****
Accuracy: 0.799		% Precision: 0.795	% Recall: 0.799		% F1_score: 0.762
---------------------------------------------------------
***** request *****
Accuracy: 0.896		% Precision: 0.893	% Recall: 0.896		% F1_score: 0.882
---------------------------------------------------------
***** offer *****
Accuracy: 0.998		% Precision: 0.995	% Recall: 0.998		% F1_score: 0.997
---------------------------------------------------------
***** aid_related *****
Accuracy: 0.777		% Precision: 0.779	% Recall: 0.777		% F1_score: 0.772
---------------------------------------------------------
***** medical_help *****
Accuracy: 0.919		% Precision: 0.896	% Recall: 0.919		% F1_score: 0.886
---------------------------------------------------------
***** medical_products *****
Accuracy: 0.954		% Precision: 0.944	% Recall: 0.954		% F1_score: 0.935
---------------------------------------------------------
***** search_and_rescue *****
Accuracy: 0.977		% Precision: 0.975	% Recall: 0.977		% F1_sc

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***** earthquake *****
Accuracy: 0.969		% Precision: 0.968	% Recall: 0.969		% F1_score: 0.968
---------------------------------------------------------
***** cold *****
Accuracy: 0.982		% Precision: 0.977	% Recall: 0.982		% F1_score: 0.973
---------------------------------------------------------
***** other_weather *****
Accuracy: 0.948		% Precision: 0.940	% Recall: 0.948		% F1_score: 0.924
---------------------------------------------------------
***** direct_report *****
Accuracy: 0.866		% Precision: 0.862	% Recall: 0.866		% F1_score: 0.844
---------------------------------------------------------


### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'classifier__estimator__n_estimators': n_estimators,
               'classifier__estimator__max_features': max_features,
               'classifier__estimator__max_depth': max_depth,
               'classifier__estimator__min_samples_split': min_samples_split,
               'classifier__estimator__min_samples_leaf': min_samples_leaf,
               'classifier__estimator__bootstrap': bootstrap}
pprint(random_grid)
# Random search of parameters, using 10 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = pipeline, param_distributions = random_grid, n_iter = 100, 
                               cv = 5, verbose = 2,scoring='f1_weighted' random_state = 42)

{'classifier__estimator__bootstrap': [True, False],
 'classifier__estimator__max_depth': [10,
                                      20,
                                      30,
                                      40,
                                      50,
                                      60,
                                      70,
                                      80,
                                      90,
                                      100,
                                      110,
                                      None],
 'classifier__estimator__max_features': ['auto', 'sqrt'],
 'classifier__estimator__min_samples_leaf': [1, 2, 4],
 'classifier__estimator__min_samples_split': [2, 5, 10],
 'classifier__estimator__n_estimators': [100,
                                         200,
                                         300,
                                         400,
                                         500,
                                       

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
rf_random.fit(X_train, y_train)

rf_random.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] classifier__estimator__n_estimators=200, classifier__estimator__min_samples_split=5, classifier__estimator__min_samples_leaf=1, classifier__estimator__max_features=sqrt, classifier__estimator__max_depth=30, classifier__estimator__bootstrap=True 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  classifier__estimator__n_estimators=200, classifier__estimator__min_samples_split=5, classifier__estimator__min_samples_leaf=1, classifier__estimator__max_features=sqrt, classifier__estimator__max_depth=30, classifier__estimator__bootstrap=True, total= 1.5min
[CV] classifier__estimator__n_estimators=200, classifier__estimator__min_samples_split=5, classifier__estimator__min_samples_leaf=1, classifier__estimator__max_features=sqrt, classifier__estimator__max_depth=30, classifier__estimator__bootstrap=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s


[CV]  classifier__estimator__n_estimators=200, classifier__estimator__min_samples_split=5, classifier__estimator__min_samples_leaf=1, classifier__estimator__max_features=sqrt, classifier__estimator__max_depth=30, classifier__estimator__bootstrap=True, total= 1.4min
[CV] classifier__estimator__n_estimators=200, classifier__estimator__min_samples_split=5, classifier__estimator__min_samples_leaf=1, classifier__estimator__max_features=sqrt, classifier__estimator__max_depth=30, classifier__estimator__bootstrap=True 
[CV]  classifier__estimator__n_estimators=200, classifier__estimator__min_samples_split=5, classifier__estimator__min_samples_leaf=1, classifier__estimator__max_features=sqrt, classifier__estimator__max_depth=30, classifier__estimator__bootstrap=True, total= 1.5min
[CV] classifier__estimator__n_estimators=200, classifier__estimator__min_samples_split=5, classifier__estimator__min_samples_leaf=1, classifier__estimator__max_features=sqrt, classifier__estimator__max_depth=30, class

...

[CV]  classifier__estimator__n_estimators=300, classifier__estimator__min_samples_split=2, classifier__estimator__min_samples_leaf=4, classifier__estimator__max_features=auto, classifier__estimator__max_depth=40, classifier__estimator__bootstrap=True, total= 2.0min
[CV] classifier__estimator__n_estimators=300, classifier__estimator__min_samples_split=2, classifier__estimator__min_samples_leaf=4, classifier__estimator__max_features=auto, classifier__estimator__max_depth=40, classifier__estimator__bootstrap=True 
[CV]  classifier__estimator__n_estimators=300, classifier__estimator__min_samples_split=2, classifier__estimator__min_samples_leaf=4, classifier__estimator__max_features=auto, classifier__estimator__max_depth=40, classifier__estimator__bootstrap=True, total= 2.0min
[CV] classifier__estimator__n_estimators=300, classifier__estimator__min_samples_split=2, classifier__estimator__min_samples_leaf=4, classifier__estimator__max_features=auto, classifier__estimator__max_depth=40, class

[CV]  classifier__estimator__n_estimators=500, classifier__estimator__min_samples_split=2, classifier__estimator__min_samples_leaf=1, classifier__estimator__max_features=auto, classifier__estimator__max_depth=10, classifier__estimator__bootstrap=False, total= 1.7min
[CV] classifier__estimator__n_estimators=500, classifier__estimator__min_samples_split=2, classifier__estimator__min_samples_leaf=1, classifier__estimator__max_features=auto, classifier__estimator__max_depth=10, classifier__estimator__bootstrap=False 
[CV]  classifier__estimator__n_estimators=500, classifier__estimator__min_samples_split=2, classifier__estimator__min_samples_leaf=1, classifier__estimator__max_features=auto, classifier__estimator__max_depth=10, classifier__estimator__bootstrap=False, total= 1.7min
[CV] classifier__estimator__n_estimators=500, classifier__estimator__min_samples_split=2, classifier__estimator__min_samples_leaf=1, classifier__estimator__max_features=auto, classifier__estimator__max_depth=10, cl

[CV]  classifier__estimator__n_estimators=1000, classifier__estimator__min_samples_split=10, classifier__estimator__min_samples_leaf=2, classifier__estimator__max_features=auto, classifier__estimator__max_depth=40, classifier__estimator__bootstrap=True, total= 6.7min
[CV] classifier__estimator__n_estimators=1000, classifier__estimator__min_samples_split=10, classifier__estimator__min_samples_leaf=2, classifier__estimator__max_features=auto, classifier__estimator__max_depth=40, classifier__estimator__bootstrap=True 
[CV]  classifier__estimator__n_estimators=1000, classifier__estimator__min_samples_split=10, classifier__estimator__min_samples_leaf=2, classifier__estimator__max_features=auto, classifier__estimator__max_depth=40, classifier__estimator__bootstrap=True, total= 6.7min
[CV] classifier__estimator__n_estimators=1000, classifier__estimator__min_samples_split=10, classifier__estimator__min_samples_leaf=2, classifier__estimator__max_features=auto, classifier__estimator__max_depth=4

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed: 3481.9min finished


Pipeline(steps=[('countVect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fd10d41b790>)),
                ('trasform', TfidfTransformer()),
                ('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=False,
                                                                        max_features='sqrt',
                                                                        n_estimators=200,
                                                                        random_state=42)))])

In [48]:
best_estimator = rf_random.best_estimator_

In [49]:
best_grid_metric = performace_metric(best_estimator, X_test, y_test)

***** related *****
Accuracy: 0.803		% Precision: 0.799	% Recall: 0.803		% F1_score: 0.770
---------------------------------------------------------
***** request *****
Accuracy: 0.899		% Precision: 0.896	% Recall: 0.899		% F1_score: 0.886
---------------------------------------------------------
***** offer *****
Accuracy: 0.998		% Precision: 0.995	% Recall: 0.998		% F1_score: 0.997
---------------------------------------------------------
***** aid_related *****
Accuracy: 0.778		% Precision: 0.777	% Recall: 0.778		% F1_score: 0.774
---------------------------------------------------------
***** medical_help *****
Accuracy: 0.919		% Precision: 0.895	% Recall: 0.919		% F1_score: 0.886
---------------------------------------------------------
***** medical_products *****
Accuracy: 0.954		% Precision: 0.941	% Recall: 0.954		% F1_score: 0.936
---------------------------------------------------------
***** search_and_rescue *****
Accuracy: 0.974		% Precision: 0.959	% Recall: 0.974		% F1_sc

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [53]:
for i, col in enumerate(y_test):
    print(col, '--- Improvement of {:0.2f}%.'.format( 100 * (best_grid_metric[i] - base_metric[i]) / base_metric[i]))

related --- Improvement of 0.55%.
request --- Improvement of 0.34%.
offer --- Improvement of 0.00%.
aid_related --- Improvement of 0.02%.
medical_help --- Improvement of 0.00%.
medical_products --- Improvement of -0.02%.
search_and_rescue --- Improvement of -0.23%.
security --- Improvement of -0.12%.
military --- Improvement of 0.04%.
child_alone --- Improvement of 0.00%.
water --- Improvement of 0.36%.
food --- Improvement of -0.04%.
shelter --- Improvement of -0.23%.
clothing --- Improvement of 0.00%.
money --- Improvement of 0.00%.
missing_people --- Improvement of 0.00%.
refugees --- Improvement of 0.00%.
death --- Improvement of 0.04%.
other_aid --- Improvement of -0.04%.
infrastructure_related --- Improvement of -0.02%.
transport --- Improvement of 0.00%.
buildings --- Improvement of -0.04%.
electricity --- Improvement of -0.02%.
tools --- Improvement of 0.00%.
hospitals --- Improvement of -0.02%.
shops --- Improvement of 0.00%.
aid_centers --- Improvement of 0.00%.
other_infrast

In [104]:
pipeline_ada = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultiOutputClassifier(
        AdaBoostClassifier(base_estimator=RandomForestClassifier(class_weight = 'balanced'), random_state = 42)
    ))
])



In [107]:
pipeline_ada.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7fd10feb18b0>)),
  ('tfidf', TfidfTransformer()),
  ('classifier',
   MultiOutputClassifier(estimator=AdaBoostClassifier(base_estimator=RandomForestClassifier(random_state=42))))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7fd10feb18b0>),
 'tfidf': TfidfTransformer(),
 'classifier': MultiOutputClassifier(estimator=AdaBoostClassifier(base_estimator=RandomForestClassifier(random_state=42))),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,

In [108]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
ada_grid = {'classifier__estimator__base_estimator__n_estimators': n_estimators,
               'classifier__estimator__base_estimator__max_features': max_features,
               'classifier__estimator__base_estimator__max_depth': max_depth,
               'classifier__estimator__base_estimator__min_samples_split': min_samples_split,
               'classifier__estimator__base_estimator__min_samples_leaf': min_samples_leaf,
               'classifier__estimator__base_estimator__bootstrap': bootstrap}
pprint(ada_grid)

{'classifier__estimator__base_estimator__bootstrap': [True, False],
 'classifier__estimator__base_estimator__max_depth': [10,
                                                      20,
                                                      30,
                                                      40,
                                                      50,
                                                      60,
                                                      70,
                                                      80,
                                                      90,
                                                      100,
                                                      110,
                                                      None],
 'classifier__estimator__base_estimator__max_features': ['auto', 'sqrt'],
 'classifier__estimator__base_estimator__min_samples_leaf': [1, 2, 4],
 'classifier__estimator__base_estimator__min_samples_split': [2, 5, 10],
 'classifier_

In [109]:
ada_random = RandomizedSearchCV(estimator = pipeline_ada, param_distributions = ada_grid, n_iter = 100, 
                               cv = 5, verbose = 2, scoring='f1_weighted', random_state = 42)

In [110]:
ada_random.fit(X_train, y_train)
ada_random.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] classifier__estimator__base_estimator__n_estimators=200, classifier__estimator__base_estimator__min_samples_split=5, classifier__estimator__base_estimator__min_samples_leaf=1, classifier__estimator__base_estimator__max_features=sqrt, classifier__estimator__base_estimator__max_depth=30, classifier__estimator__base_estimator__bootstrap=True 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  classifier__estimator__base_estimator__n_estimators=200, classifier__estimator__base_estimator__min_samples_split=5, classifier__estimator__base_estimator__min_samples_leaf=1, classifier__estimator__base_estimator__max_features=sqrt, classifier__estimator__base_estimator__max_depth=30, classifier__estimator__base_estimator__bootstrap=True, total=57.6min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 57.6min remaining:    0.0s


[CV] classifier__estimator__base_estimator__n_estimators=200, classifier__estimator__base_estimator__min_samples_split=5, classifier__estimator__base_estimator__min_samples_leaf=1, classifier__estimator__base_estimator__max_features=sqrt, classifier__estimator__base_estimator__max_depth=30, classifier__estimator__base_estimator__bootstrap=True 
[CV]  classifier__estimator__base_estimator__n_estimators=200, classifier__estimator__base_estimator__min_samples_split=5, classifier__estimator__base_estimator__min_samples_leaf=1, classifier__estimator__base_estimator__max_features=sqrt, classifier__estimator__base_estimator__max_depth=30, classifier__estimator__base_estimator__bootstrap=True, total=61.7min
[CV] classifier__estimator__base_estimator__n_estimators=200, classifier__estimator__base_estimator__min_samples_split=5, classifier__estimator__base_estimator__min_samples_leaf=1, classifier__estimator__base_estimator__max_features=sqrt, classifier__estimator__base_estimator__max_depth=30,

KeyboardInterrupt: 

In [62]:
ADA_grid_metric = performace_metric(ada_grid, X_test, y_test)

***** related *****
Accuracy: 0.745		% Precision: 0.644	% Recall: 0.745		% F1_score: 0.648
---------------------------------------------------------
***** request *****
Accuracy: 0.831		% Precision: 0.691	% Recall: 0.831		% F1_score: 0.754
---------------------------------------------------------
***** offer *****
Accuracy: 0.998		% Precision: 0.995	% Recall: 0.998		% F1_score: 0.996
---------------------------------------------------------
***** aid_related *****
Accuracy: 0.588		% Precision: 0.559	% Recall: 0.588		% F1_score: 0.446
---------------------------------------------------------
***** medical_help *****
Accuracy: 0.918		% Precision: 0.842	% Recall: 0.918		% F1_score: 0.878
---------------------------------------------------------
***** medical_products *****
Accuracy: 0.952		% Precision: 0.907	% Recall: 0.952		% F1_score: 0.929
---------------------------------------------------------
***** search_and_rescue *****
Accuracy: 0.975		% Precision: 0.951	% Recall: 0.975		% F1_sc

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.989		% Precision: 0.978	% Recall: 0.989		% F1_score: 0.984
---------------------------------------------------------
***** earthquake *****
Accuracy: 0.912		% Precision: 0.876	% Recall: 0.912		% F1_score: 0.870
---------------------------------------------------------
***** cold *****
Accuracy: 0.981		% Precision: 0.963	% Recall: 0.981		% F1_score: 0.972
---------------------------------------------------------
***** other_weather *****
Accuracy: 0.948		% Precision: 0.898	% Recall: 0.948		% F1_score: 0.922
---------------------------------------------------------
***** direct_report *****
Accuracy: 0.810		% Precision: 0.657	% Recall: 0.810		% F1_score: 0.725
---------------------------------------------------------


In [91]:
for i, col in enumerate(y_test):
    print(col, '--- Improvement of {:0.2f}%.'.format( 100 * (ADA_grid_metric[i] - base_metric[i]) / base_metric[i]))

related --- Improvement of -6.78%.
request --- Improvement of -7.22%.
offer --- Improvement of -0.02%.
aid_related --- Improvement of -24.33%.
medical_help --- Improvement of -0.15%.
medical_products --- Improvement of -0.18%.
search_and_rescue --- Improvement of -0.16%.
security --- Improvement of -0.02%.
military --- Improvement of -0.02%.
child_alone --- Improvement of 0.00%.
water --- Improvement of -1.42%.
food --- Improvement of -4.82%.
shelter --- Improvement of -2.00%.
clothing --- Improvement of -0.08%.
money --- Improvement of -0.04%.
missing_people --- Improvement of -0.02%.
refugees --- Improvement of 0.00%.
death --- Improvement of -0.42%.
other_aid --- Improvement of -0.09%.
infrastructure_related --- Improvement of 0.00%.
transport --- Improvement of -0.18%.
buildings --- Improvement of -0.34%.
electricity --- Improvement of -0.04%.
tools --- Improvement of 0.00%.
hospitals --- Improvement of 0.00%.
shops --- Improvement of 0.00%.
aid_centers --- Improvement of 0.00%.
ot

### 9. Export your model as a pickle file

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(cv2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.